In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [1]:
source('../0_parameters.sh')

source('../PRS_PheWAS.functions.R')

source('MR_functions.R')


In [2]:
ukb_array_combined_annot_f %>%
fread(select=c('ID', 'ld_indep')) -> annot_df


In [40]:
HGI_cc <- 'C2'
HGI_sx <- 'eur_leave_ukbb_23andme'
GBE_ID <- 'BIN_FC11006152' #Blood clot or DVT diagnosed by doctor
p_thr <- 5e-8

In [4]:
file.path(data_d, 'plink_format_UKB_cal', UKB_cal_f) %>%
str_replace('@@HGI_case_control@@', HGI_cc) %>%
str_replace('@@HGI_suffix@@', HGI_sx) %>%
read_sumstats() -> hgi_df


In [5]:
file.path(data_scratch_d, 'UKB_MR', 'sumstats', sprintf('%s.p1e-5.tsv.gz', GBE_ID)) %>%
read_sumstats() %>%
filter(log10P < log10(p_thr)) -> ukb_df


In [6]:
if(! 'BETA' %in% names(ukb_df)){
    ukb_df %>%
    mutate(BETA = log(OR)) %>%
    rename('SE'='LOG(OR)_SE') -> ukb_df
}


In [7]:
ukb_df %>%
filter(
    ID %in% (annot_df %>% filter(ld_indep == T) %>% pull(ID))
) %>%
join_ukb_hgi(hgi_df) -> df


In [8]:
df %>% dim()

[1] 32 26

In [9]:
bind_rows(
    lm(BETA_hgi ~ 0 + BETA_ukb, df, weights = (df$SE_hgi ** (-2))) %>% fit_to_df() %>%
    mutate(method = 'IVW')
) %>%
mutate(
    HGI_case_control = HGI_cc,
    HGI_suffix = HGI_sx,
    trait = GBE_ID
) 
# %>%
# select(HGI_case_control, HGI_suffix, trait, method, variable, estimate, SE, z_or_t_value, P)

variable,estimate,SE,z_or_t_value,P,method,HGI_case_control,HGI_suffix,trait
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
BETA_ukb,0.07440127,0.02350923,3.164768,0.003467648,IVW,C2,eur_leave_ukbb_23andme,BIN_FC11006152


In [16]:
plot_beta_vs_beta <- function(df, lmfit){
    df %>% ggplot(aes(x = BETA_ukb, y = BETA_hgi)) + 
    geom_abline(
        slope = (lmfit)$coefficients[['BETA_ukb']],
        color='red', alpha=.5
    ) + 
    geom_point(alpha=.1) +
    theme_bw()
}


In [10]:
lmfit <- lm(BETA_hgi ~ 0 + BETA_ukb, df, weights = (df$SE_hgi ** (-2)))

In [27]:
fit_to_df(lmfit)

variable,estimate,SE,z_or_t_value,P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
BETA_ukb,0.0629811,0.02339049,2.692594,0.01224045


In [11]:
summary(lmfit) 


Call:
lm(formula = BETA_hgi ~ 0 + BETA_ukb, data = df, weights = (df$SE_hgi^(-2)))

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-4.4391 -1.2815 -0.5375  0.1114  1.8607 

Coefficients:
         Estimate Std. Error t value Pr(>|t|)   
BETA_ukb  0.07440    0.02351   3.165  0.00347 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.466 on 31 degrees of freedom
Multiple R-squared:  0.2442,	Adjusted R-squared:  0.2198 
F-statistic: 10.02 on 1 and 31 DF,  p-value: 0.003468


In [12]:
compute_IVW_estimate(df)

[1] 0.06533104

In [27]:
sprintf('%.2e', (lmfit)$coefficients[['BETA_ukb']])

[1] "7.44e-02"

In [ ]:
df %>% ggplot(aes(x = BETA_ukb, y = BETA_hgi)) + 
geom_abline(
    slope = (lmfit)$coefficients[['BETA_ukb']],
    color='red', alpha=.5
) + 
geom_errorbar( aes(ymin = BETA_hgi-SE_hgi,  ymax = BETA_hgi+SE_hgi),  alpha=.2) +
geom_errorbarh(aes(xmin = BETA_ukb-SE_ukb,  xmax = BETA_ukb+SE_ukb),  alpha=.2) +
geom_point(alpha=.5) +
theme_bw(base_size=16) + 
labs(
    title = sprintf(
        'MR (%s)\nIVW estimate: %.2e, p = %.2e',
        HGI_sx,
        (lmfit)$coefficients[['BETA_ukb']],
        fit_to_df(lmfit) %>% pull(P)
    ),
    y = sprintf('BETA[SE] HGI (%s)', HGI_cc),
    x = sprintf('BETA[SE] UKB (%s)', GBE_ID)
) -> p


In [41]:
df %>% ggplot(aes(x = BETA_ukb, y = BETA_hgi)) + 
geom_abline(
    slope = (lmfit)$coefficients[['BETA_ukb']],
    color='red', alpha=.5
) + 
geom_errorbar( aes(ymin = BETA_hgi-SE_hgi,  ymax = BETA_hgi+SE_hgi),  alpha=.2) +
geom_errorbarh(aes(xmin = BETA_ukb-SE_ukb,  xmax = BETA_ukb+SE_ukb),  alpha=.2) +
geom_point(alpha=.5) +
theme_bw(base_size=16) + 
labs(
    title = sprintf(
        'MR (%s)\nIVW estimate: %.2e, p = %.2e',
        HGI_sx,
        (lmfit)$coefficients[['BETA_ukb']],
        fit_to_df(lmfit) %>% pull(P)
    ),
    y = sprintf('BETA[SE] HGI (%s)', HGI_cc),
    x = sprintf('BETA[SE] UKB (%s)\n%s', GBE_ID, 'Blood clot or DVT diagnosed by doctor')
) -> p


In [43]:
ggsave(sprintf('%s.%s.%s.png', HGI_cc, HGI_sx, GBE_ID), p, height=8, width=8)
ggsave(sprintf('%s.%s.%s.pdf', HGI_cc, HGI_sx, GBE_ID), p, height=8, width=8)